In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
import time

In [ ]:
start = time.time()

In [ ]:
#List of Products
excel_products = pd.read_csv('C:/Users/guilh_1irk1to/Desktop/Thesis Code/Numerical Experiment/Products_Curvecatch_Numerical_Experiment.csv')

In [ ]:
#Products
products = excel_products['ProductID'].to_list()

#Revenue Generated by each product 
prod_revenue = dict(zip(excel_products['ProductID'],excel_products['Price']))

#Cost associated with each product 
prod_cost = dict(zip(excel_products['ProductID'],excel_products['Cost']))

#Mapping model type with each product
model_type_products = excel_products.groupby('Model Type')['ProductID'].apply(list).reset_index()

#Mapping tier price with each product
price_tier_products = excel_products.groupby('Price Tier')['ProductID'].apply(list).reset_index()

#Mapping Vendor group  with each product
vendor_groups = excel_products.groupby('Vendor Group')['ProductID'].apply(list).reset_index()

#Mapping Permanent/Fashion with each product
color_clusters = excel_products.groupby('Color Cluster')['ProductID'].apply(list).reset_index()

#Mapping sizes with each product
bra_sizes = excel_products.groupby('Size')['ProductID'].apply(list).reset_index()

#Mapping sizes with each product
variant_products = excel_products.groupby('VariantID')['ProductID'].apply(list).reset_index()

In [ ]:
#List of Customers + Preferences + Customer Type

#Sub Experiment 2
excel_customers = pd.read_csv("C:/Users/guilh_1irk1to/Desktop/Thesis Code/Numerical Experiment/Sub_exp_two/Single_Choice_Customer_Preferences_subexp_2.csv")

#Sub Experiment 1
#excel_customers = pd.read_csv("C:/Users/guilh_1irk1to/Desktop/Thesis Code/Numerical Experiment/Sub_exp_one/Single_Choice_Customer_Preferences_subexp_1.csv")

In [ ]:
#Customers
clients = excel_customers['Customer'].to_list()
#Customer Type
customer_type = dict(zip(excel_customers['Customer'],excel_customers['Customer Type']))

In [ ]:
#List of Preferences
excel_preferences = excel_customers.drop(columns=['Customer Type'])
list_pref = excel_preferences.set_index('Customer').T.to_dict('list')

In [ ]:
#Cardinality - Max amount of different SKUS in the assortment
diff_skus = 130

In [ ]:
#Matching supply and demand
matching_demand =[]
revenue = []
for customer, pref in list_pref.items():
    for i in pref:
        matching_demand.append((i,customer))
        revenue.append(prod_revenue[i])
#print(matching_demand)

In [ ]:
#Adding Variables
model = gp.Model('toy_example') 

#if product is available or not
x = model.addVars(products, obj= prod_revenue, vtype = GRB.BINARY, name= 'x')

#flow
flow = model.addVars(matching_demand, obj=prod_revenue,vtype = GRB.BINARY, name="flow")

In [ ]:
#Upper Bound Capacity Constraint
model.addConstr(sum(x[i] for i in products) <= diff_skus)
model.update()

In [ ]:
#constraint for each customer taking into account their customer type
model.addConstrs((gp.quicksum(flow.select("*", customer)) <= customer_type[customer]
                                  for customer in clients), name="customer")
model.update()

In [ ]:
#Constraint that allows a customer only to choose a product if it is available in the assortment
product_available = model.addConstrs(((flow.select(d[0], d[1])[0] if flow.select(d[0], d[1]) else 0) <= x[d[0]] for d in matching_demand), name="available")
model.update()

In [ ]:
#Constraint that forces the products that are chosen by the customer to be under their preferences  (not the revenue they generate)
for customer, pref in list_pref.items():
    for i in range(1, len(pref)):
        for ii in range(0, i):
            prev_item = pref[ii]
            item = pref[i]
            model.addConstr(flow.select(prev_item, customer)[0]>= flow.select(item, customer)[0]*x[prev_item], f'preferences{customer}_{item}')

In [ ]:
model.update()

In [ ]:
model.setObjective(gp.quicksum((flow.select(item, customer)[0] if flow.select(item, customer) else 0)*prod_revenue[item]
                   for item in products for customer in clients),GRB.MAXIMIZE)

In [ ]:
model.optimize()

In [ ]:
for v in model.getVars():
        print('%s %g' % (v.VarName, v.X))

In [ ]:
end = time.time()